In [1]:
import nltk
import random
import os
import json
from nltk.stem import  PorterStemmer
import pickle

In [2]:
def load_dataset():  
    dataset=[]
    for root, dirs, files in os.walk('../Dataset'):
        for directory in dirs:
            for root_f, dirs_f, files_f in os.walk(os.path.join(root,directory)):
                for each_file in files_f:
                    with open(os.path.join(root,directory)+'/'+each_file) as f:
                        data_temp=json.load(f)    
                    dataset+=[(sentence,each_file) for sentence in data_temp]
    return dataset

In [3]:
#initialising porterstemmer
ps=PorterStemmer()

In [4]:
def to_unigram(data):
    dataset=[]
    dataset+=[([items for items in sentence.split()],polarity) for sentence,polarity in data]    
    return dataset    

In [5]:
def get_words(x):    #returns all the words in the  data set
    vocab = []
    for sentence,polarity in x:  
            vocab.extend(sentence.split())
    return vocab

In [6]:
#execution
labeled_data=load_dataset()
vocab=set(get_words(labeled_data))
vocab=[ps.stem(word) for word in vocab]
labeled_data=to_unigram(labeled_data)

In [7]:
#bigrams
print labeled_data[10]

([u'mani', u'new', u'thing'], 'sli_pos')


In [8]:
print len(labeled_data)

201378


In [9]:
print len(vocab)

6544


In [10]:
def extract_features(document):       #each sentences contains a list of words and true/false 
    features = {}
    for word in vocab:
         features['contains(%s)' % word] = (word in document)
    return features        

In [11]:
train_test = nltk.classify.apply_features(extract_features, labeled_data)

In [12]:
train_set=train_test[:-10000]
test_set=train_test[-10000:]

In [19]:
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [ ]:
nltk.classify.accuracy(classifier, test_set)

In [28]:
sample='poor performance'
classifier.classify(extract_features(sample.split()))

'sli_neg'

In [24]:
print classifier.show_most_informative_features(25)

Most Informative Features
       contains(horribl) = True              neg : pos    =   1957.1 : 1.0
       contains(terribl) = True              neg : neutra =   1565.5 : 1.0
          contains(bore) = True              neg : pos    =   1428.4 : 1.0
           contains(bad) = True              neg : pos    =   1254.9 : 1.0
      contains(unfortun) = True              neg : pos    =    657.3 : 1.0
          contains(hard) = True           sli_ne : pos    =    559.7 : 1.0
      contains(everyday) = True           sli_ne : pos    =    519.0 : 1.0
          contains(sick) = True              neg : pos    =    460.5 : 1.0
           contains(due) = True           sli_ne : pos    =    455.2 : 1.0
        contains(tediou) = True              neg : sli_po =    428.6 : 1.0
         contains(heavi) = True           sli_ne : pos    =    408.8 : 1.0
         contains(littl) = True           sli_ne : pos    =    395.1 : 1.0
         contains(great) = True              pos : neg    =    387.4 : 1.0

In [23]:
#save  the model
with open('naive_bayes_model','w') as f:
    pickle.dump(classifier,f)

In [14]:
import pickle
with open('naive_bayes_model') as f:
    classifier=pickle.load(f)

In [18]:
sample='a effective learning experience'
classifier.classify(extract_features(sample.split()))

'pos'

In [ ]:
nltk.classify.accuracy(classifier, test_set)